# Stationarity Checker — YoY then ADF

Ce notebook applique **YoY (%)** à toutes tes séries, puis teste la **stationnarité (ADF)**.

**Sorties :**

- Un tableau *verdict par variable* + stats ADF

- Un fichier Excel `rapport_stationnarite_yoy.xlsx` avec 2 feuilles : `report` et `non_stationnaires`

- Un DataFrame `df_yoy` prêt pour la suite du pipeline


⚙️ **À renseigner** dans la cellule suivante : chemin du fichier et nom de la colonne de date.


In [ ]:
# === Configuration : À ADAPTER ===
DATA_PATH = 'Data/finalData.csv'     
DATE_COL  = 'date'           
SHEET     = None             # <- si Excel, nom de feuille ou None (1ère feuille)
FREQ      = 'MS'             # <- fréquence attendue pour l'index Datetime: 'MS' = début de mois
TRAIN_START = '2013-06-01'
TRAIN_END = '2020-03-01'

# Seuils & options ADF
ALPHA     = 0.05             # seuil de décision
MIN_OBS   = 36               # nb min d'observations pour lancer ADF 
AUTO_LAG  = 'AIC'            # choix du critère d'autolag : 'AIC', 'BIC', 't-stat', None


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from statsmodels.tsa.stattools import adfuller

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


In [ ]:
# Gestion des données manquantes avec de modèles AR

import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from typing import Dict, Tuple

def _select_p_aic(y: pd.Series, pmax: int = 6, nom= None) -> Tuple[int, float]:
    """
    Sélectionne l'ordre p d'un AR(p) par AIC sur une série stationnaire.
    Ne teste que p <= len(y) - 2 pour éviter les sur-ajustements.
    """
    y = y.dropna()
    if len(y) < 10:
        # série trop courte -> AR(1) par défaut
        return 1, np.inf
    
    best_p, best_aic = 1, np.inf
    upper = int(min(pmax, max(1, len(y) - 2)))
    for p in range(1, upper + 1):
        try:
            model = ARIMA(y, order=(p, 0, 0), trend=("c" if nom and nom in I_0_c else "n"))
            res = model.fit()
            if np.isfinite(res.aic) and res.aic < best_aic:
                best_aic, best_p = res.aic, p
        except Exception:
            continue
    return best_p, best_aic


import numpy as np
import pandas as pd
from typing import Callable, Iterable, Optional
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

def yoy_pct(s: pd.Series, periods: int = 12) -> pd.Series:
    """Taux de croissance en glissement annuel (en %)"""
    return 100 * (s / s.shift(periods) - 1)

def select_p_default(s: pd.Series, pmax: int = 6) -> int:
    """
    Sélecteur AR(p) minimaliste si tu n’as pas encore _select_p_aic.
    Retourne p=1 si pas assez d'observations.
    """
    s = s.dropna()
    n = len(s)
    if n < 20:
        return 1
    best_p, best_aic = 1, np.inf
    for p in range(1, min(pmax, max(1, n // 4)) + 1):
        try:
            aic = ARIMA(s, order=(p, 0, 0), trend='c').fit().aic
            if aic < best_aic:
                best_aic, best_p = aic, p
        except Exception:
            pass
    return best_p


def ar_impute_series(
    s: pd.Series,
    p_select_fn: Callable[[pd.Series], int],
    trend: str = 'c',
    min_obs: int = 20,
    only_ragged: bool = False,
) -> pd.Series:
    """
    Impute les NaN de s en avançant dans le temps:
    - À chaque NaN à t, on ajuste AR(p) sur s[:t-1].dropna() (si assez d'obs),
      puis on prédit s[t] (one-step). Si plusieurs NaN consécutifs,
      on boucle et on réutilise les prévisions comme 'observations' pour avancer.
    - if only_ragged=True, on n’impute que la 'queue' (NaN en fin d’échantillon).
    - trend: 'c' (constante) ou 'n' (sans constante).

    Retourne une copie de s imputée (les points non imputables restent NaN).
    """
    s_imp = s.copy()
    idx = s_imp.index

    # Déterminer les positions à imputer
    if only_ragged:
        # on ne prend que les NaN après la dernière valeur observée
        last_obs = s_imp.last_valid_index()
        if last_obs is None:
            return s_imp  # rien à faire
        to_impute = idx.get_indexer_for(idx[idx.get_loc(last_obs) + 1:])
    else:
        to_impute = np.where(s_imp.isna())[0]

    for pos in to_impute:
        # détermine la fenêtre d'estimation: tout jusqu'à pos-1
        est_end_pos = pos - 1
        if est_end_pos < 0:
            continue
        y_est = s_imp.iloc[:est_end_pos + 1].dropna()
        if len(y_est) < min_obs:
            # pas assez de données -> on laisse NaN 
            continue

        # sélectionner p sur données dispo avant t
        try:
            p = p_select_fn(y_est)
            p = max(1, int(p))
        except Exception:
            p = 1

        # ajuste AR(p) et prédit s[pos]
        try:
            res = ARIMA(y_est, order=(p, 0, 0), trend=trend, enforce_stationarity=False).fit()
            yhat = res.forecast(steps=1).iloc[0]
            s_imp.iloc[pos] = yhat
        except Exception:
            # en dernier recours, on ne remplit pas
            pass

    return s_imp

def preprocessing(
    df: pd.DataFrame,
    periods: int = 12,
    log_col: Optional[Iterable[str]] = None,
    diff_col: Optional[Iterable[str]] = None,
    yoy_skip: Optional[Iterable[str]] = None,   # colonnes à NE PAS passer en YoY 
    ragged: bool = False,                       # n’imputer que la queue ? 
    ar_only_trend_c: bool = True,               # 'c' sinon 'n'
    p_select_fn: Optional[Callable[[pd.Series], int]] = None,
    min_obs_for_ar: int = 20,
) -> pd.DataFrame:
    """
    Pipeline simple:
    1) log sur log_col
    2) YoY (%) sur toutes les colonnes sauf yoy_skip
    3) diff() sur diff_col
    4) Remplacement des inf -> NaN
    5) Imputation AR itérative colonne par colonne
    """
    if log_col is None: log_col = []
    if diff_col is None: diff_col = []
    if yoy_skip is None: yoy_skip = []

    if p_select_fn is None:
        p_select_fn = select_p_default

    out = df.copy()

    # 1) LOG (vérifier positivité)
    for col in log_col:
        if col in out.columns:
            # Si des valeurs <= 0, les mettre à NaN 
            bad = out[col] <= 0
            if bad.any():
                out.loc[bad, col] = np.nan
            out[col] = np.log(out[col])

    # 2) YoY (%) — sauf colonnes à skipper  
    for col in out.columns:
        if col in yoy_skip:
            continue
        out[col] = yoy_pct(out[col], periods=periods)

    # 3) Diff (si nécessaire por stationnariser) 
    for col in diff_col:
        if col in out.columns:
            out[col] = out[col].diff()

    # 4) Inf -> NaN (et tracer les colonnes concernées)
    has_inf = out.replace([np.inf, -np.inf], np.nan)
    inf_cols = out.columns[out.isin([np.inf, -np.inf]).any()]
    for col in inf_cols:
        n_inf = np.isinf(out[col]).sum()
        print(f"[INFO] Colonne '{col}' contient {n_inf} infinis -> remplacés par NaN")
    out = has_inf

    # 5) Imputation AR itérative
    trend = 'c' if ar_only_trend_c else 'n'
    imputed = {}
    for col in out.columns:
        s = out[col]
        # si toute la colonne est NaN, on la laisse telle quelle
        if s.notna().sum() == 0:
            imputed[col] = s
            continue
        s_imp = ar_impute_series(
            s=s,
            p_select_fn=p_select_fn,
            trend=trend,
            min_obs=min_obs_for_ar,
            only_ragged=ragged
        )
        imputed[col] = s_imp

    out_imp = pd.DataFrame(imputed, index=out.index)

    return out_imp


In [4]:
def fillna_with_annual_mean(
    df: pd.DataFrame,
    value_col: str,
) -> pd.DataFrame:
    """
    Remplit les NaN de `value_col` par la moyenne annuelle (calculée via `date_col`).
    - `date_col` doit être convertible en datetime (format libre).
    - La moyenne est propre à chaque année civile.
    """
    out = df.copy()

    # Sécuriser le type datetime
    out.index = pd.to_datetime(out.index, errors="coerce")

    # Extraire l'année
    year = out.index.year

    # Calcul de la moyenne annuelle par groupe
    annual_mean = out.groupby(year)[value_col].transform("mean")

    # Remplissage uniquement là où c'est NaN
    out[value_col] = out[value_col].fillna(annual_mean)

    return out


## 1) Chargement & nettoyage

In [5]:
def load_any(path, sheet=None):
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"Fichier introuvable: {path.resolve()}")
    if path.suffix.lower() in ['.xlsx', '.xls']:
        return pd.read_excel(path, sheet_name=sheet)
    if path.suffix.lower() == '.csv':
        return pd.read_csv(path)
    if path.suffix.lower() == '.parquet':
        return pd.read_parquet(path)
    raise ValueError('Format non supporté: utilisez .xlsx, .csv ou .parquet')

def sanitize_columns(df):
    # Nettoie les noms de colonnes (accents, espaces, caractères spéciaux)
    cols = (df.columns.astype(str)
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('ascii')
        .str.replace(r"[^0-9a-zA-Z_]+", '_', regex=True)
        .str.replace(r"__+", '_', regex=True)
        .str.strip('_')
    )
    df = df.copy()
    df.columns = cols
    # Retire doublons éventuels
    df = df.loc[:, ~df.columns.duplicated()]
    return df

raw = load_any(DATA_PATH, SHEET)
raw = sanitize_columns(raw)

# Construit un DatetimeIndex propre
raw[DATE_COL] = pd.to_datetime(raw[DATE_COL], errors='coerce')
raw = raw.dropna(subset=[DATE_COL]).sort_values(DATE_COL).reset_index(drop=True)
raw = raw.set_index(DATE_COL)

# Reindex mensuel (optionnel mais recommandé), ne comble pas les manquants
idx = pd.date_range(start=raw.index.min(), end=raw.index.max(), freq=FREQ)
df  = raw.reindex(idx)

print(df.shape, 'lignes/colonnes =', df.shape)
df.head()


(192, 48) lignes/colonnes = (192, 48)


,cacao,imp,CPI,ipc_alim,ipc_habill,ipc_ameub,ipc_loge,ipc_sante,ipc_transport,ipc_com,ipc_hotel,tib_credit_total,tib_credit_particuliers,tib_credit_snf_pub,tib_credit_assurance_fonds,tib_credit_snf_priv,tib_credit_autres_inst_depots,tib_credit_personnel_banque,tib_credit_export,tib_credit_tresorerie,tib_credit_duree_1_3m,tib_credit_duree_1_2a,tib_credit_duree_5_10a,tib_depot_total,tib_depot_admin_centrale,tib_depot_particuliers,tib_depot_imf,tib_depot_assurance_fonds,tib_depot_snf_priv,tib_depot_entreprise_indiv,tib_depot_personnel_banque,tib_depot_terme,tib_depot_epargne_special,tib_depot_bons_caisse,tib_depot_duree_1_3m,tib_depot_duree_3_6m,tib_depot_duree_6m_1a,tib_depot_duree_1_2a,monnaie_circulation_fiduciaire,monnaie_circ_encaisses_tresor,Stage,argent,mean_ntl_dnb,mean_evi,mean_vhi,mean_ndvi,mean_no2,mean_pcp
2010-01-01,3522.095398,11950934.0,93.009342,86.32,85.53,86.82,91.71,101.54,95.95,108.80,85.97,7.72,12.60,7.90,NaN,8.10,12.4,4.60,8.30,7.40,6.80,12.00,8.70,5.40,5.60,5.70,5.80,5.90,5.20,5.30,6.30,5.50,3.40,5.70,5.70,5.00,5.80,4.80,1301.629,2.361,23.0,16.0,NaN,2862.586601,0.487643,5315.692302,0.001182,15.709527
2010-02-01,3276.545775,13772708.0,92.859525,86.12,85.11,86.37,91.58,100.69,96.36,108.04,85.56,7.67,13.10,14.00,15.0,7.73,12.0,3.18,8.62,7.25,6.99,12.25,5.56,5.26,4.23,5.17,5.49,7.71,5.12,5.33,6.07,5.24,3.46,5.54,5.15,5.13,4.99,5.26,1263.079,2.784,12.0,0.0,NaN,3141.226744,0.631357,5659.445928,0.000590,69.412171
2010-03-01,3089.645814,19267924.0,92.977965,87.05,85.26,86.45,92.10,100.96,96.65,107.88,85.78,7.92,12.81,9.63,NaN,7.72,NaN,3.62,8.10,7.39,7.24,12.85,7.40,5.38,4.53,6.23,5.93,6.24,5.20,5.19,6.06,5.37,3.37,5.78,5.39,5.11,5.32,6.35,1226.821,2.818,19.0,0.0,NaN,3738.110109,0.655786,5861.910968,0.000465,106.907115
2010-04-01,3221.237212,16702679.0,97.551669,91.60,85.02,86.45,92.71,101.98,97.85,107.67,86.38,7.75,13.03,NaN,14.0,8.21,NaN,3.17,9.02,7.14,6.70,11.73,9.70,5.19,5.57,5.08,5.38,5.70,5.05,5.31,5.88,5.15,3.46,5.64,4.71,5.05,5.00,5.43,1238.997,4.417,24.0,0.0,NaN,4291.416169,0.555524,6378.322339,0.000398,115.475326
2010-05-01,3165.761094,16843754.0,97.768090,91.82,84.18,85.23,92.94,101.19,98.12,107.60,86.88,7.46,13.18,9.00,NaN,7.83,NaN,3.53,9.40,6.84,6.26,11.31,11.88,5.20,5.36,5.10,5.11,6.62,5.18,5.24,6.28,5.14,3.09,5.94,5.26,5.32,5.06,4.58,1233.998,2.478,18.0,0.0,NaN,4766.519719,0.533095,6859.272722,0.000426,138.074379


In [ ]:
df_yoy = df.copy()
df_ = df.copy()

df_yoy = preprocessing(
    df=df_yoy,                 # DataFrame mensuel brut (avant standardisation)
    periods=12,
    yoy_skip=['inflation'],     # inflation est déjà un taux (%), donc pas de YoY
    ragged= False,              
    ar_only_trend_c=True,       # AR avec constante
    p_select_fn=_select_p_aic,  
    min_obs_for_ar=12
)

df_yoy.describe()

[INFO] Colonne 'tib_credit_snf_pub' contient 4 infinis -> remplacés par NaN
[INFO] Colonne 'tib_credit_assurance_fonds' contient 12 infinis -> remplacés par NaN
[INFO] Colonne 'tib_credit_autres_inst_depots' contient 8 infinis -> remplacés par NaN
[INFO] Colonne 'tib_credit_export' contient 8 infinis -> remplacés par NaN
[INFO] Colonne 'tib_depot_admin_centrale' contient 14 infinis -> remplacés par NaN
[INFO] Colonne 'tib_depot_imf' contient 6 infinis -> remplacés par NaN
[INFO] Colonne 'tib_depot_personnel_banque' contient 6 infinis -> remplacés par NaN
[INFO] Colonne 'tib_depot_bons_caisse' contient 7 infinis -> remplacés par NaN
[INFO] Colonne 'monnaie_circ_encaisses_tresor' contient 1 infinis -> remplacés par NaN
[INFO] Colonne 'argent' contient 7 infinis -> remplacés par NaN


,cacao,imp,CPI,ipc_alim,ipc_habill,ipc_ameub,ipc_loge,ipc_sante,ipc_transport,ipc_com,ipc_hotel,tib_credit_total,tib_credit_particuliers,tib_credit_snf_pub,tib_credit_assurance_fonds,tib_credit_snf_priv,tib_credit_autres_inst_depots,tib_credit_personnel_banque,tib_credit_export,tib_credit_tresorerie,tib_credit_duree_1_3m,tib_credit_duree_1_2a,tib_credit_duree_5_10a,tib_depot_total,tib_depot_admin_centrale,tib_depot_particuliers,tib_depot_imf,tib_depot_assurance_fonds,tib_depot_snf_priv,tib_depot_entreprise_indiv,tib_depot_personnel_banque,tib_depot_terme,tib_depot_epargne_special,tib_depot_bons_caisse,tib_depot_duree_1_3m,tib_depot_duree_3_6m,tib_depot_duree_6m_1a,tib_depot_duree_1_2a,monnaie_circulation_fiduciaire,monnaie_circ_encaisses_tresor,Stage,argent,mean_ntl_dnb,mean_evi,mean_vhi,mean_ndvi,mean_no2,mean_pcp
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,174.000000,174.000000,165.000000,152.000000,174.000000,169.000000,174.000000,172.000000,174.000000,174.000000,174.000000,174.000000,174.000000,172.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,174.000000,180.000000,180.000000,180.000000,172.000000,153.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,12.338568,13.983790,3.026852,2.165835,1.462431,1.577137,1.768119,0.408469,0.988160,-0.174180,1.225054,-0.781612,3.514797,27.685820,-13.018296,-0.514500,-4.396208,9.955122,3.641136,0.023153,0.308764,4.143140,1.233403,-0.400713,-4.411775,-0.038515,-8.031936,0.342202,-0.368604,0.846163,-3.122969,-0.359796,3.179566,-1.574791,0.146732,0.502620,0.270813,0.487364,9.559137,-2.225465,1.502217,12.101795,21.581109,0.848248,0.137376,0.678121,-3.321377,6.445592
std,40.804738,98.961694,4.818070,7.057158,3.198862,3.233946,4.028190,1.702512,4.110211,2.982263,5.124474,8.883651,53.568958,312.410569,37.795878,12.599534,62.551711,54.044126,63.267561,16.075867,16.340059,56.115482,35.224186,7.349357,49.045981,10.972489,37.118661,11.434487,10.865148,16.745551,29.269752,7.483146,26.434285,31.543767,15.936142,15.699376,11.397655,16.278312,8.369351,22.828033,30.776180,24.839965,142.967940,10.277319,11.825166,10.317842,25.511478,45.555899
min,-36.294902,-99.999110,-14.012171,-27.214080,-10.941926,-12.119622,-19.525986,-6.314409,-15.116088,-13.882777,-16.894977,-32.028470,-86.573311,-100.000000,-100.000000,-43.567961,-100.000000,-63.961814,-151.543577,-57.326892,-53.485064,-88.161209,-82.084691,-27.131783,-100.000000,-23.895028,-100.000000,-32.035398,-37.888199,-35.097002,-100.000000,-27.111984,-63.431542,-100.000000,-46.121593,-39.152542,-31.389365,-34.309623,-6.713027,-100.000000,-75.000000,-46.153846,-1344.684041,-16.354198,-40.907259,-16.484886,-83.089519,-77.727563
25%,-6.677428,-18.470016,0.161700,0.279002,0.929400,0.314580,0.759548,-0.567797,-0.539799,-1.238019,0.468329,-5.423219,-9.199655,-14.427861,-17.028257,-6.741564,-25.300000,-26.067790,-13.067307,-8.004355,-7.155826,-12.468836,-11.433220,-3.856759,-16.669149,-6.269943,-18.930940,-5.331613,-6.215224,-8.018596,-14.846587,-3.843075,-6.279796,-8.301570,-8.419374,-7.750084,-7.444758,-9.373388,3.305236,-2.935501,-15.000000,-4.276490,-9.045928,-3.640401,-6.818744,-3.421769,-14.053435,-20.114887
50%,3.027837,3.219868,2.379337,2.545925,1.708027,1.686826,2.120756,0.495727,0.926081,0.060779,1.401895,-0.350518,-1.065598,3.205128,-7.948654,0.697154,-4.199351,5.665706,2.902038,-0.442886,0.397823,-3.238211,-1.352166,-1.035885,-1.887502,-0.003198,-5.214727,0.264418,-0.351966,0.966587,-2.898822,-0.735039,0.281339,-1.568416,0.180198,0.359902,0.380196,-0.523158,8.063206,0.000000,0.000000,9.878049,11.918474,0.643007,0.305869,-0.050318,-3.476895,-1.408890
75%,15.145065,31.289526,5.689499,5.685742,2.869250,3.003959,3.792986,1.250076,2.880378,1.091010,3.158041,4.112620,6.193154,27.667521,0.696360,6.331405,10.248902,30.369762,14.121242,7.075295,7.052758,8.813754,6.182978,2.5

In [12]:
missing = {}
df_yoy = df_yoy[df_yoy.index >= TRAIN_START]
for col in df_yoy.columns:
    nb = df_yoy[col].isna().sum()
    if nb>0:
        missing[col] = nb
if missing != {}: 
    print("Les colonnes avec valeurs manquantes à partir du début de la période de Train: ")
    for key, val in missing.items():
        print(f"{key} a {val} na")

In [13]:
# Remplacer par la moyenne anuelle pour ce qui reste

df_yoy = fillna_with_annual_mean(df_yoy,'tib_credit_assurance_fonds')


In [ ]:
df_yoy[df_yoy.index <= TRAIN_END].to_csv("DataYoY.csv",index=False)

In [16]:
df_adf = pd.read_csv("adf_sequentiel_5pct_20250903-060408.csv")
df_adf.head()

,series,ok,status,diff_order,final_type,selected_lags,tau_stat,tau_cv_5pct,phi_name,phi_stat,phi_cv_5pct,phi_keep
0,cacao,True,stationary,0,none,6,-2.667150,-1.95,NaN,NaN,NaN,NaN
1,imp,True,stationary,0,trend,6,-4.477879,-3.45,phi3,10.036017,6.49,True
2,CPI,True,stationary,0,none,6,-3.073496,-1.95,NaN,NaN,NaN,NaN
3,ipc_alim,True,stationary,0,none,6,-2.769298,-1.95,NaN,NaN,NaN,NaN
4,ipc_habill,True,stationary,0,none,6,-2.555766,-1.95,NaN,NaN,NaN,NaN


In [17]:
diff_col = {}
for row in range(0,df_adf.shape[0]):
    if df_adf.iloc[row,:]['diff_order'] > 0:
        diff_col[df_adf.iloc[row,:]['series']] = df_adf.iloc[row,:]['diff_order']
diff_col

{'ipc_transport': 1, 'ipc_com': 1, 'tib_depot_entreprise_indiv': 1}

In [19]:
for key, val in diff_col.items():
    i=1
    while i<=val:
        df_yoy[key] = df_yoy[key].diff()
        i +=1

In [20]:
df_yoy.reset_index(inplace=True)
df_yoy.rename(columns={'index':'date'}, inplace=True)
df_yoy.head()

,date,cacao,imp,CPI,ipc_alim,ipc_habill,ipc_ameub,ipc_loge,ipc_sante,ipc_transport,ipc_com,ipc_hotel,tib_credit_total,tib_credit_particuliers,tib_credit_snf_pub,tib_credit_assurance_fonds,tib_credit_snf_priv,tib_credit_autres_inst_depots,tib_credit_personnel_banque,tib_credit_export,tib_credit_tresorerie,tib_credit_duree_1_3m,tib_credit_duree_1_2a,tib_credit_duree_5_10a,tib_depot_total,tib_depot_admin_centrale,tib_depot_particuliers,tib_depot_imf,tib_depot_assurance_fonds,tib_depot_snf_priv,tib_depot_entreprise_indiv,tib_depot_personnel_banque,tib_depot_terme,tib_depot_epargne_special,tib_depot_bons_caisse,tib_depot_duree_1_3m,tib_depot_duree_3_6m,tib_depot_duree_6m_1a,tib_depot_duree_1_2a,monnaie_circulation_fiduciaire,monnaie_circ_encaisses_tresor,Stage,argent,mean_ntl_dnb,mean_evi,mean_vhi,mean_ndvi,mean_no2,mean_pcp
0,2013-06-01,0.783099,-18.133256,1.786769,1.785010,7.769704,3.864631,1.838197,-1.161290,NaN,NaN,1.352598,-3.351206,17.622951,-33.333333,-37.487669,-9.383378,-8.259587,-25.862069,9.677419,-0.630915,-4.785479,-11.572942,-5.739514,1.714286,1.663202,-4.000000,10.503282,-13.078150,4.044118,NaN,11.932773,1.333333,5.491329,-6.176962,2.226721,-7.241379,-20.326679,11.835749,1.603536,-19.318182,21.428571,50.000000,701.237129,-1.188123,-9.697505,0.258496,1.035239,-18.271229
1,2013-07-01,-1.757119,-25.728331,1.052590,1.050668,7.797553,3.587730,2.226364,-1.529295,NaN,NaN,0.971072,-4.865557,-3.522337,7.920792,-37.487669,-9.701493,59.651669,-29.234973,11.970534,-3.661972,-17.372263,-15.571284,-28.915663,-3.759398,-16.849015,-6.500000,-29.078014,-10.363636,-10.073260,NaN,12.854758,-3.759398,0.000000,-0.175439,-2.189781,1.465798,8.750000,-17.391304,3.113222,16.671646,-21.052632,-9.090909,-6.148007,1.481184,-2.636299,3.962610,-21.325662,-24.093655
2,2013-08-01,-1.140570,-30.155236,1.262356,1.266324,7.573912,2.592749,1.801802,-2.529528,1.504478,0.050528,0.631429,-11.216216,-5.518946,-51.800000,-37.487669,-11.983471,18.992806,170.000000,-86.363636,-14.328808,-11.018364,6.717850,-10.635697,4.466019,46.954315,3.867403,-41.569282,-14.444444,11.328125,22.547537,16.354344,4.854369,0.000000,2.131439,6.340580,-14.503817,27.680798,10.428305,3.464349,-37.870144,0.000000,-20.000000,27.854180,-3.593093,-0.198347,2.200222,-12.240863,-14.461771
3,2013-09-01,-0.161675,-4.236855,1.543689,1.542468,6.836196,3.126311,1.197110,-2.305192,-0.049528,-0.221788,0.182500,-5.781866,-30.355666,40.182281,-41.769231,-5.508475,3.669725,5.541562,-11.307692,-2.000000,1.157025,-88.161209,-82.084691,-2.857143,3.970223,-0.558659,24.434389,-0.701754,-2.214022,30.099114,34.057971,-2.666667,0.000000,-3.098107,-10.192308,30.522088,14.588235,-4.393305,4.955107,-37.870144,12.500000,33.333333,-32.569157,0.640601,-4.765755,-1.806869,10.247313,-4.863994
4,2013-10-01,10.844640,52.281766,-0.096920,-0.099334,6.681271,3.800800,1.161357,-1.403578,0.248779,-0.704237,0.467955,-12.545676,-86.573311,4.833333,-33.206107,-13.137755,-90.421456,-26.476578,21.296834,-10.489510,-11.145511,-2.213115,-25.951904,10.898662,11.890838,-4.401408,-4.142012,0.541516,16.831683,-36.363316,9.426987,12.810707,1.139601,-22.260274,2.071563,-27.113238,1.101322,-0.423729,6.683625,-37.954422,-15.384615,37.500000,31.190189,-5.779159,-8.207223,-5.512477,-17.762738,-35.871625


In [ ]:
df_yoy.to_csv("Data/DataStationary.csv",index=False)

In [ ]:
# df_yoy.set_index('date').to_csv("Data/DataStationary2.csv",index=False)